In [1]:
import sys
import json

sys.path.append('../../../data/corpus-reader/')

from benchmark_reader import Benchmark
from benchmark_reader import select_files

# extract reference texts
b = Benchmark()
b.fill_benchmark([('../../../data/en/test',
                   'rdf-to-text-generation-test-data-with-refs-en.xml')])
b.b2json('../eval_metadata/', 'table_data_en.json')

In [2]:
html_triplet = '''

  <tr>
    <td class="tg-0lax">SUBJ</td>
    <td class="tg-0lax">PRED</td>
    <td class="tg-0lax">OBJ</td>
  </tr>

'''

In [3]:
html_head = '''

<div class="center">
<table class="tg">

<thead>
  <tr>
    <th class="tg-0pky"><b>Subject</b></th>
    <th class="tg-0pky"><b>Predicate</b></th>
    <th class="tg-0pky"><b>Object</b></th>
  </tr>
</thead>

<tbody>
REPLACE_DATA
</tbody>

</table>
</div>

'''

In [4]:
with open('../eval_metadata/sampled_ids_en.txt', 'r') as f2:
    sampled_ids = f2.read().splitlines()
with open('../eval_metadata/table_data_en.json', 'r') as f1:
    table_data = json.load(f1)['entries']

In [5]:
# get items for the sampled ids
sampled_table_data = []
for item in table_data:
    for key, value in item.items():
        if key in sampled_ids:
            sampled_table_data.append(item)

In [6]:
# prepare triples
triplet_data = {}

# save info about triple size
triple_size = {}

for entry in sampled_table_data:
    lexs = []
    rows = []
    for k, v in entry.items():
        n = 0
        triples = v['modifiedtripleset']       
        for t in triples:
            subj = t['subject'].replace('_', ' ')
            pred = t['property'].replace('_', ' ')
            obj = t['object'].replace('_', ' ')
            this_t = [subj, pred, obj]
            rows.append(this_t)
            n += 1
        triple_size[k] = n
                
    triplet_data[k] = []
    triplet_data[k].append(rows)

In [7]:
# main loop, construct xml tables
qual_triplets_vis = {}
for sample_id, triplet_set in triplet_data.items():
    xml_set = ''
    for this_set in triplet_set:
        for item in this_set:
            subj, pred, obj = item
            this_xml_triplet = html_triplet.replace('SUBJ', subj).replace('PRED', pred).replace('OBJ', obj)
            xml_set += this_xml_triplet
    full_xml_table = html_head.replace('REPLACE_DATA', xml_set)
    qual_triplets_vis[sample_id] = full_xml_table

In [8]:
with open('../data/en/tables/tables_xml.json', 'w') as f0:
    json.dump(qual_triplets_vis, f0)